In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

from radial.batchflow import Dataset, FilesIndex, Pipeline, action, inbatch_parallel, B, V, C
import radial.batchflow as ds
from radial.batchflow.models.tf import TFModel, VGG7
from radial import RadialBatch

In [2]:
train_index = FilesIndex(path='../data/reconsidered/train_npz/*.npz', no_ext=True)
test_index = FilesIndex(path='../data/reconsidered/test_npz/*.npz', no_ext=True)

train_set = Dataset(train_index, batch_class=RadialBatch)
test_set = Dataset(test_index, batch_class=RadialBatch)

In [3]:
norm_ppl = (Pipeline()
            .load(fmt='npz', components=['time', 'derivative', 'rig_type', 'target'])
            .drop_negative()
            .normalize(src=['time', 'derivative'], dst=['norm_time', 'norm_derivative'],\
                       dst_range=[None, 'derivative_range'])
            .normalize(src='target', dst='norm_target', src_range='derivative_range')
           )

draw_ppl = norm_ppl << train_set

# save images

In [ ]:
for i in range(len(train_set.indices)):
    btch = draw_ppl.next_batch(1)
    plt.figure(figsize=(15, 10))
    plt.scatter(btch.norm_time[0], btch.norm_derivative[0], s=20)
    plt.savefig('./train_images/'+ btch.indices[0] + '.png', dpi=100, bbox_inches='tight')

In [5]:
plt.ioff()

In [6]:
draw_ppl = norm_ppl << test_set

for i in range(len(test_set.indices)):
    btch = draw_ppl.next_batch(1)
    plt.figure(figsize=(15, 10))
    plt.scatter(btch.norm_time[0], btch.norm_derivative[0], s=20)
    plt.savefig('./test_images/'+ btch.indices[0] + '.png', dpi=100, bbox_inches='tight')

# save labels

In [20]:
draw_ppl = norm_ppl << test_set
draw_ppl.reset_iter()
btch = draw_ppl.next_batch(len(test_set.indices))

table1 = pd.DataFrame(np.vstack([btch.indices, btch.norm_target]).T, columns=['index', 'target'])
# table.to_csv('test_targets.csv', index=False)

In [21]:
draw_ppl = norm_ppl << train_set

draw_ppl.reset_iter()
btch = draw_ppl.next_batch(len(train_set.indices))

table2 = pd.DataFrame(np.vstack([btch.indices, btch.norm_target]).T, columns=['index', 'target'])
# table.to_csv('train_targets.csv', index=False)

In [23]:
len(table1)

90

In [32]:
table3 = pd.concat([table1, table2], axis=0, join='outer')

In [33]:
len(table3)

401

In [39]:
table3.to_csv('all_targets.csv', index=False)

# convert to greyscale:

In [43]:
import os
from PIL import Image

for item in os.listdir('test_images/'):
    Image.open('test_images/' + item).convert('L').save('bw_test_images/' + item)